In [1]:
import numpy as np
import pandas as pd 
import json

In [2]:
df = pd.read_csv('r490.csv')
df = df[df['r1_charge_heater'] >= 0]
X = np.array(df[['r1_temp', 'r2_temp', 'r1_pressure', 'r2_pressure']])
Y = np.array(df[['r1_charge_heater', 'process_ron', 'process_yield']])

In [6]:
files = [
    {
        'path' : 'preds_from_traces/noiseless_easy_eg.npz',
        'name' : 'Easy Constraints - Noiseless',
    },
    {
        'path' : 'preds_from_traces/noisy_easy_eg.npz',
        'name' : 'Easy Constraints - Noisy',
    },
    {
        'path' : 'preds_from_traces/noisy_easy_constrained_eg.npz',
        'name' : 'Easy Constraints - Noisy Restricted Jumps',
    },
    {
        'path' : 'preds_from_traces/noiseless_hard_ei.npz',
        'name' : 'Difficult Constraints - Noiseless',
    }
]
results = []
for file in files:
    d = np.load(file['path'],allow_pickle=True)
    yhats = np.array(d['yhats'])
    exp_cfg = d['exp_cfg'].tolist()
    feasible_ix = (Y[:,1] >= exp_cfg['min_ron']) & (Y[:,2] >= exp_cfg['min_yield'])
    print("Feasibles: %d out of %d (%0.2f)" % (np.sum(feasible_ix), df.shape[0], np.sum(feasible_ix) / df.shape[0]))
    
    feasible_indecies = set(np.where(feasible_ix)[0])

    true_min = np.min(Y[feasible_ix,0])

    ch_diffs = []
    for yhat in yhats:

        if len(yhat.shape) > 2:
            yhat = np.mean(yhat, axis=0)

        predicted_feasible_ix = (yhat[:,1] >= exp_cfg['min_ron']) & (yhat[:,2] >= exp_cfg['min_yield'])

        predicted_min_feasible_ix = np.argmin(yhat[:,0] * predicted_feasible_ix + 
                                              1e9 * (1-predicted_feasible_ix))

        predicted_min_feasible = Y[predicted_min_feasible_ix,0]

        if predicted_min_feasible_ix in feasible_indecies:
            ch_diffs.append(predicted_min_feasible / true_min)
    print("Minimum is actually feasible: %d" % len(ch_diffs))
    print("Ratio of predicted minimum to actual minimum feasible: %0.2f" % np.mean(ch_diffs))
    
    ch_diffs_random = []
    for r in range(100):

        rand_feasible_ix = np.random.choice(list(feasible_indecies))

        ch = Y[rand_feasible_ix, 0]

        ch_diffs_random.append(ch / true_min) 
    print("Random feasible ratio: %0.2f" % np.mean(ch_diffs))
    
    results.append({
        "scenario" : file['name'],
        "n_feasible" : len(ch_diffs),
        "mean_ratio" : np.mean(ch_diffs),
        "mean_random_ratio" : np.mean(ch_diffs_random)
    })
output_df = pd.DataFrame(results)
output_df

Feasibles: 490 out of 8950 (0.05)
Minimum is actually feasible: 46
Ratio of predicted minimum to actual minimum feasible: 7.34
Random feasible ratio: 7.34
Feasibles: 490 out of 8950 (0.05)
Minimum is actually feasible: 46
Ratio of predicted minimum to actual minimum feasible: 7.83
Random feasible ratio: 7.83
Feasibles: 490 out of 8950 (0.05)
Minimum is actually feasible: 45
Ratio of predicted minimum to actual minimum feasible: 21.23
Random feasible ratio: 21.23
Feasibles: 75 out of 8950 (0.01)
Minimum is actually feasible: 12
Ratio of predicted minimum to actual minimum feasible: 1.02
Random feasible ratio: 1.02


,scenario,n_feasible,mean_ratio,mean_random_ratio
0,Easy Constraints - Noiseless,46,7.339356,152.688527
1,Easy Constraints - Noisy,46,7.826328,139.733107
2,Easy Constraints - Noisy Restricted Jumps,45,21.225592,140.706220
3,Difficult Constraints - Noiseless,12,1.023469,1.346192


In [7]:
output_df['p_feasible'] = output_df['n_feasible'] / 50
output_df

,scenario,n_feasible,mean_ratio,mean_random_ratio,p_feasible
0,Easy Constraints - Noiseless,46,7.339356,152.688527,0.92
1,Easy Constraints - Noisy,46,7.826328,139.733107,0.92
2,Easy Constraints - Noisy Restricted Jumps,45,21.225592,140.706220,0.90
3,Difficult Constraints - Noiseless,12,1.023469,1.346192,0.24


In [8]:
output_df.to_csv("tmp/table3.csv", index=False)